# Vehicle Route Planning

## Packages

In [26]:
## Data Acess and Manipulation
import json
import numpy as np
import pandas as pd
import snowflake.connector as sf

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## for vehicle route planning
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

## Utilities
import os
import logging
from functools import partial
#from google.colab import userdata
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor as Executor

## Geospatial Work
import folium
from folium import plugins
import osmnx as ox
import networkx as nx
from shapely.geometry import shape, Point
from math import sin, cos, sqrt, atan2, radians

## Multiprocessing
from multiprocessing import Pool, cpu_count

## Utilities
### importing necessary functions from dotenv library and loading variables from .env file
import os
from dotenv import load_dotenv 
load_dotenv()

# Geopandas
import geopandas

from tqdm import tqdm

import plotly.express as px

In [27]:
# Notebook Configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [28]:
#>> Contains layout and callbacks for app 1
''' 
Notes:
This file is for creating app 1, it ontains layout and callbacks for app 1
'''
# Import packages---------------------------------------------------------------------------------------------
## Data Acess and Manipulation
import numpy as np

## Import App Packages
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
from datetime import datetime as dt
from dash.exceptions import PreventUpdate
import pandas as pd
import io
from flask import send_file

## Data Visualization
import plotly.graph_objs as go

##Local Imports
from src.utils.data_loaders import get_data_from_sf, get_geojson, load_dict
from src.utils.queries import data_query_order_line, data_query_item_weights
from src.utils.data_processing import data_preprocessing, calculate_metrics, generate_vehicle_names, data_preprocessing_map
from src.utils.map import maps, route_maps
from src.utils.vehicle_route_planning import DistanceCalculator, create_data_model, run_model
from src.utils.routes_processing import enhance_optimized_route

from app import app

## Utilities
import os
from dotenv import load_dotenv 
load_dotenv()

# Incorporate data---------------------------------------------------------------------------------------------

True

In [29]:
# Weights Master
df_weights = pd.read_excel(
    r'assets\data\Item Weight.xlsx'
    )

print(f"Shape of the imported dataset is: {df_weights.shape}")
df_weights.head(5)

Shape of the imported dataset is: (677, 2)


,itemCode,itemWeight
0,GOSF0001,10.6
1,DBRF0006,0.1
2,AEHG0009,1.0
3,GRND0001,0.4
4,ULPL0014,1.0


In [30]:
# Loading Data
df_order_line = get_data_from_sf(data_query_order_line)
#df_weights = get_data_from_sf(data_query_item_weights)
df_order_line = df_order_line[df_order_line['customerCode']!='17658'].copy() 
df_order_line['itemTotalWeight'] = df_order_line['itemTotalWeight'].replace(0, 1)
    
js = get_geojson(os.getenv("geojson_loc_2"))

# Preprocessing the Data
df = data_preprocessing(df_order_line, df_weights, js)

choropleth_df, order_details = data_preprocessing_map(df)

## Exploration

In [32]:
# Gallega Cold Store
start = [24.931690466392208, 55.06185223067843]
order_details = df.groupby(['customerCode', 'customerName', 'customerAddress', 'customerRegion', 'customerLat', 'customerLong']
                            ).agg({
                                'externalDocumentNo': lambda x: pd.Series.nunique(x),
                                'itemTotalWeight': 'sum',
                                'itemCode': lambda x: pd.Series.nunique(x),
                                'invoiceQuantity': 'sum',
                                'saleValue': 'sum',
                                'costValue': 'sum'
                                }
                            ).reset_index().copy()
order_details.rename(columns={
                            'externalDocumentNo': 'totalOrders',
                            'itemTotalWeight': 'totalWeight',
                            'itemCode': 'totalItems'
                            }, 
                    inplace=True
                    )
order_details['invoiceQuantity'] = order_details['invoiceQuantity'].abs()
order_details['costValue'] = order_details['costValue'].abs()
order_details['grossProfit'] = order_details['saleValue'] - order_details['costValue']
order_details['grossProfitMargin'] = order_details['grossProfit'] / order_details['saleValue'] * 100
order_details['customerRegion'] = order_details['customerRegion'].str.upper()
order_details['totalWeight'] = order_details['totalWeight'].fillna(0) # Replace NaN values in the itemTotalWeight column with 0

# Adding base row
base_row = {'customerCode': 0000,'customerName': 'base', 'customerAddress': 'base', 'customerRegion': 'base', 'customerLat': start[0], 'customerLong': start[1]}
order_details = pd.concat([pd.DataFrame(base_row, index=[0]), order_details], ignore_index=True)
order_details["base"] = order_details["customerName"].apply(lambda x: 1 if x=='base' else 0)

# Replace NaN values in the itemTotalWeight column with 0
order_details['totalWeight'].fillna(0, inplace=True)
order_details.head()


class DistanceCalculator:
    def __init__(self, bbox=None, network_type="drive"):
        self.G = None
        self.bbox = bbox
        self.network_type = network_type

    def create_graph(self):
        if self.bbox is None:
            raise ValueError("Bounding box is not set.")
        north, south, east, west = self.bbox
        self.G = ox.graph_from_bbox(north, south, east, west, network_type=self.network_type)
        self.G = ox.add_edge_speeds(self.G)
        self.G = ox.add_edge_travel_times(self.G)
        logger.info("Graph created")

    def calculate_duration(self, a, b, units='travel_time', duration=True):
        if self.G is None:
            self.create_graph()
        if a == b:
            return 0
        try:
            route = nx.shortest_path(self.G, source=a, target=b, method='dijkstra', weight=units)
            attrs = ox.utils_graph.route_to_gdf(self.G, route)
        except nx.NetworkXNoPath:
            return np.nan
        except Exception as e:
            logger.error(f"Error {e} occurred for inputs {a}, {b}")
            return np.nan

        return attrs[units].sum() if duration else attrs['length'].sum()

    @staticmethod
    def haversine_dist( x, y):
        # Approximate radius of earth in km
        R = 6373.0

        lat1 = radians(float(x[0]))
        lon1 = radians(float(x[-1]))
        lat2 = radians(float(y[0]))
        lon2 = radians(float(y[-1]))

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c

        return distance

    @staticmethod
    def calculate_gd(args):
        x, y = args
        return DistanceCalculator.haversine_dist(x, y)

    def pairwise_distance(self, args, method='GD'):
        p1, p2 = args
        if method == 'GD':
            return self.calculate_gd((p1, p2))
        elif method == 'GRAPH':
            return self.calculate_duration(p1, p2)

    def df_to_dm(self, df, lat_col, long_col, uid, demand_ref, base_flag, method='GD'):
        if not isinstance(df, pd.DataFrame):
            raise ValueError("df must be a pandas DataFrame")

        required_cols = [lat_col, long_col, uid, demand_ref, base_flag]
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"DataFrame must contain columns: {required_cols}")

        xy_coordinates = list(map(lambda x: list(x), df[[lat_col, long_col]].values))
        xy_ids = df[uid].to_list()
        xy_demand = list(df[demand_ref].values.flatten())
        xy_demand[0] = 0

        max_workers = cpu_count()
        chunksize = 100  # Experiment with this value

        if method == 'GD':
            all_pairs = [(p1, p2) for p1 in xy_coordinates for p2 in xy_coordinates]
        elif method == 'GRAPH':
            if self.G is None:
                self.create_graph()
            xy_coordinates = list(map(lambda x: ox.distance.nearest_nodes(self.G, x[1], x[0]), xy_coordinates))
            all_pairs = [(p1, p2) for p1 in xy_coordinates for p2 in xy_coordinates]

        #with Executor(max_workers=max_workers) as executor:
        dm = list(map(partial(self.pairwise_distance, method=method), all_pairs))
        dm_matrix = np.array(dm).reshape(len(xy_coordinates), len(xy_coordinates))

        # Normalizing data for optumizer
        dm_matrix = dm_matrix*100
        dm_matrix = dm_matrix.astype(int)
        xy_demand = list(map(int, xy_demand))

        logger.info(f"Average distance between the points: {dm_matrix.mean()}")
        logger.info(f"Range distance between the points: {dm_matrix.min()} - {dm_matrix.max()}")

        return xy_ids, xy_coordinates, xy_demand, dm_matrix

# Usage
bbox = (22.4969475367, 26.055464179, 51.5795186705, 56.3968473651)  # Set only if using 'GRAPH' method
distance_calculator = DistanceCalculator(bbox=bbox, network_type="drive")
id, coordinates, demand, dm= distance_calculator.df_to_dm(order_details, 'customerLat', 'customerLong', 'customerCode', 'totalWeight', 'base', 'GD')

C:\Users\lakshmi.n.GAGROUP\AppData\Local\Temp\ipykernel_6216\2520114857.py:33: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [33]:
"""# Distance Matrics
bbox = (22.4969475367, 26.055464179, 51.5795186705, 56.3968473651)  # Set only if using 'GRAPH' method
distance_calculator = DistanceCalculator(bbox=bbox, network_type="drive")
id, coordinates, demand, dm = distance_calculator.df_to_dm(order_details, 'customerLat', 'customerLong', 'customerCode', 'totalWeight', 'base', 'GD')
"""

'# Distance Matrics\nbbox = (22.4969475367, 26.055464179, 51.5795186705, 56.3968473651)  # Set only if using \'GRAPH\' method\ndistance_calculator = DistanceCalculator(bbox=bbox, network_type="drive")\nid, coordinates, demand, dm = distance_calculator.df_to_dm(order_details, \'customerLat\', \'customerLong\', \'customerCode\', \'totalWeight\', \'base\', \'GD\')\n'

In [8]:
data = create_data_model(demand, dm)
manager, routing, solution, output= run_model(data)

## Constraint Optimization

In [34]:
def get_num_trucks(total_weight):
    # Calculate the ideal weights based on a 70-30 mix
    weight_3ton = 0.7 * total_weight
    weight_1ton = 0.3 * total_weight

    # Calculate the number of trucks, rounding to the nearest whole number
    num_3ton_trucks = round(weight_3ton / 3000)
    num_1ton_trucks = round(weight_1ton / 1000)

    # Calculate the total weight these trucks can carry
    total_carried_weight = (num_3ton_trucks * 3000) + (num_1ton_trucks * 1000)

    # Adjusting for the remainder if total carried weight is less than total weight
    while total_carried_weight < total_weight:
        # Check which addition gets closer to the goal
        if (total_weight - total_carried_weight) >= 3000:
            num_3ton_trucks += 1
            total_carried_weight += 3000
        else:
            num_1ton_trucks += 1
            total_carried_weight += 1000

    return [3000] * num_3ton_trucks + [1000] * num_1ton_trucks + [1000]*1

truck_weights = get_num_trucks(sum(demand))
len(truck_weights) , sum(truck_weights)

(3, 5000)

In [35]:
truck_weights

[3000, 1000, 1000]

In [36]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    vehicle_capacities = get_num_trucks(sum(demand))
    data["distance_matrix"] = dm
    data["demands"] = demand
    data["vehicle_capacities"] = vehicle_capacities
    data["num_vehicles"] = len(vehicle_capacities)
    data["depot"] = 0
    return data

In [37]:

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    routes_dict = {}
    #print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        route_seq=[]
        route_seq_dict = {}
        index = routing.Start(vehicle_id)
        # plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_distance_list = []
        route_load = 0
        route_load_list = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            # plan_output += f" {node_index} Load({route_load}) -> "
            route_seq.append(node_index)
            route_load_list.append(route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        # plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        # plan_output += f"Distance of the route: {route_distance}m\n"
        # plan_output += f"Load of the route: {route_load}\n"
        # print(plan_output)
        total_distance += route_distance
        total_load += route_load
        route_seq_dict['route_plan'] = route_seq
        route_seq_dict['cumulative_route_load'] = route_load_list
        # route_seq_dict['route_distance'] = route_distance_list
        routes_dict[vehicle_id] =  route_seq_dict
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    return routes_dict


In [38]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [39]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)
search_parameters.time_limit.FromSeconds(30)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    logger.info("Solution found!")
    output = print_solution(data, manager, routing, solution)
else:
    logger.info("No solution found!")

Total distance of all routes: 85518m
Total load of all routes: 3489


In [40]:
print(solution)

Assignment(Capacity0 (0) | Capacity1 (301) | Capacity2 (1707) | Capacity3 (639) | Capacity4 (1739) | Capacity5 (1742) | Capacity6 (640) | Capacity7 (1746) | Capacity8 (175) | Capacity9 (120) | Capacity10 (1306) | Capacity11 (1715) | Capacity12 (668) | Capacity13 (168) | Capacity14 (915) | Capacity15 (1315) | Capacity16 (2179) | Capacity17 (180) | Capacity18 (2186) | Capacity19 (674) | Capacity20 (2627) | Capacity21 (1899) | Capacity22 (2844) | Capacity23 (177) | Capacity24 (2169) | Capacity25 (1026) | Capacity26 (1060) | Capacity27 (1108) | Capacity28 (1292) | Capacity29 (2847) | Capacity30 (1527) | Capacity31 (1740) | Capacity32 (1777) | Capacity33 (2845) | Capacity34 (2796) | Capacity35 (1419) | Capacity36 (2472) | Capacity37 (2365) | Capacity38 (963) | Capacity39 (0) | Capacity40 (1318) | Capacity41 (2838) | Capacity42 (617) | Capacity43 (2826) | Capacity44 (1406) | Capacity45 (679) | Capacity46 (616) | Capacity47 (1112) | Capacity48 (1741) | Capacity49 (2282) | Capacity50 (2504) | 

In [41]:
output = print_solution(data, manager, routing, solution)

Total distance of all routes: 85518m
Total load of all routes: 3489


In [42]:
route_output = enhance_optimized_route(output, coordinates, id)

API call failed for this list of coordinates coordinates:  [[55.06185223067843, 24.931690466392208], ['54.7657693', '24.6602292'], ['54.7575763', '24.6707191'], ['54.3747856', '24.50100336'], ['54.370499', '24.4931406'], ['54.3597598', '24.4871026'], ['54.36914106', '24.47915454'], ['54.36944566', '24.4779622'], ['54.3677833333333', '24.4728616666666'], ['54.36736939', '24.47635665'], ['54.366818333333335', '24.478395'], ['54.3590733333333', '24.4795116666666'], ['54.35046', '24.4766288'], ['54.33945', '24.467705'], ['54.3522282', '24.4653153'], ['54.36076666666666', '24.459656666666667'], ['54.38882814', '24.45850988'], ['54.3905066666666', '24.448175'], ['54.3975683333333', '24.4440783333333'], ['54.40946454', '24.44114063'], ['54.41577136', '24.4432398'], ['54.4311572', '24.4331815'], ['54.49636961', '24.37160745'], ['54.48985735', '24.35997755'], ['54.47736356', '24.34921475'], ['54.4699459', '24.3404444'], ['54.4787668', '24.3414307'], ['54.49656478', '24.34999375'], ['54.5372453'

## Getting routes

In [19]:
import requests
import json
import time

def reverse_coordinates(input):
  output = []
  for each in input:
    each.reverse()
    output.append(each)
  return output

def return_coords(coords,ls):
    r_ls = [coords[each] for each in ls]
    r_ls = list(map(list,r_ls))
    r_ls = reverse_coordinates(input=r_ls)
    return r_ls

def return_ids(ids,ls):
    r_ls = [ids[each] for each in ls]
    return r_ls

def call_api(coordinates):

  body = {"coordinates": coordinates}

  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': '5b3ce3597851110001cf624822b79fbe37204f6a83490d4a1566d9a5',
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post('https://api.openrouteservice.org/v2/directions/driving-car', json=body, headers=headers)

  # print(call.status_code, call.reason)
  if call.status_code == 200:
    response = json.loads(call.text)
    geometry = response['routes'][0]['geometry']
    distance = round(response['routes'][0]['summary']['distance']/1000,1)
    duration = round(response['routes'][0]['summary']['duration']/60,1)
  else:
    print("API call failed for this list of coordinates coordinates: ", coordinates)
    print(call.text)
    geometry = None
    distance = None
    duration = None

  return geometry, distance, duration

def enhance_optimized_route(op, coords, ids):
  output = {}
  for k, v in op.items():
    output[k] = {}
    for k2, v2 in v.items():
      time.sleep(2)
      if len(return_coords(coords ,op[k]['route_plan'])) > 1:
        api_response = call_api(coordinates=return_coords(coords, op[k]['route_plan']))
        output[k]['route_geometry'] = api_response[0]
        output[k]['route_distance'] = api_response[1]
        output[k]['route_duration'] = api_response[2]
      else:
        output[k]['route_geometry'] = None
        output[k]['route_distance'] = None
        output[k]['route_duration'] = None

      output[k]['route_plan'] = op[k]['route_plan']
      output[k]['cumulative_route_load'] = op[k]['cumulative_route_load']
      output[k]['route_distance'] = 0
      output[k]['route_coords'] = return_coords(coords, op[k]['route_plan'])
      output[k]['route_ids'] = return_ids(ids, op[k]['route_plan'])

  return output

In [20]:
route_output = enhance_optimized_route(output, coordinates, id)

In [21]:
route_output_df=pd.DataFrame.from_dict(route_output, orient='index')

In [22]:
route_output_df[
    [
        'route_distance', 'route_duration', 'route_plan',
        'cumulative_route_load', 'route_coords', 'route_ids'
        ]
    ].head(2)
route_output_df.head()

,route_geometry,route_distance,route_duration,route_plan,cumulative_route_load,route_coords,route_ids
0,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,355.2,"[0, 111, 110, 62, 35, 8, 51, 116, 48, 53, 25, ...","[0, 88, 114, 116, 159, 186, 210, 212, 225, 315...","[[55.06185223067843, 24.931690466392208], [54....","[0, 9075, 9031, 20221994, 17370, 11093, 20106,..."
1,None,0,NaN,[0],[0],"[[55.06185223067843, 24.931690466392208]]",[0]
2,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,626.1,"[0, 80, 82, 39, 72, 106, 57, 86, 75, 47, 63, 6...","[0, 1, 3, 3, 3, 4, 13, 15, 15, 17, 20, 22, 28,...","[[55.06185223067843, 24.931690466392208], [55....","[0, 3020, 3169, 17927, 20223098, 825, 20221695..."
3,skdwCqkaoI}@m@kCiBa@a@[e@?QH[pHqNGEyCwBsAeAmAq...,0,108.7,"[0, 78, 38, 90, 105, 61, 66, 16, 76, 20, 7, 24]","[0, 4, 8, 18, 20, 23, 29, 389, 391, 394, 398, ...","[[55.06185223067843, 24.931690466392208], [55....","[0, 2599, 17815, 472, 820, 20221909, 20222440,..."


In [23]:
route_output_df['route_coords'][1]
print(route_output_df['route_geometry'][1])

None


In [24]:
from openrouteservice import convert
decoded = convert.decode_polyline(route_output_df['route_geometry'][3])
m = folium.Map(location=start, tiles="Cartodb Positron", zoom_start=12)
folium.GeoJson(decoded).add_to(m)
m

In [25]:
print(decoded)

{'type': 'LineString', 'coordinates': [[55.06249, 24.9313], [55.06272, 24.93161], [55.06325, 24.93231], [55.06342, 24.93248], [55.06361, 24.93262], [55.0637, 24.93262], [55.06384, 24.93257], [55.06633, 24.93104], [55.06636, 24.93108], [55.06696, 24.93185], [55.06731, 24.93227], [55.06772, 24.93266], [55.06814, 24.93299], [55.06855, 24.93327], [55.06895, 24.93351], [55.07103, 24.93448], [55.07148, 24.93471], [55.07238, 24.93518], [55.07319, 24.93563], [55.07325, 24.93564], [55.07356, 24.93581], [55.07382, 24.93594], [55.07404, 24.93602], [55.075, 24.93629], [55.07533, 24.93639], [55.07564, 24.93651], [55.07597, 24.93654], [55.07617, 24.93649], [55.07627, 24.9365], [55.07647, 24.93656], [55.07656, 24.93661], [55.07664, 24.93668], [55.0767, 24.93677], [55.07674, 24.93686], [55.07676, 24.93695], [55.07675, 24.93711], [55.07672, 24.93721], [55.07667, 24.93729], [55.0766, 24.93737], [55.07652, 24.93743], [55.07642, 24.93748], [55.07629, 24.93751], [55.07615, 24.93752], [55.07602, 24.93749], 

In [26]:
from openrouteservice import convert

def html_popup_series(row):
    try:
        html = f"""
            <h3> Customer Details </h3><br>
            <ul>
            <li>Customer Code = {row['customerCode'].values[0]}</li>
            <li>Customer Address = {row['customerAddress'].values[0]}</li>
            <li>Total Orders = {row['totalOrders'].values[0]}</li>
            <li>Total Order weight = {row['totalWeight'].values[0]}</li>
            <li>Total Items = {row['totalItems'].values[0]}</li>
            <li>Sale Value = {row['saleValue'].values[0]}</li>
            <li>Gross Profit Margin = {row['grossProfitMargin'].values[0]}</li>
            </ul>
            """
        return html
    except IndexError:
        return None

i = 1
# Locals
coordinates = route_output_df['route_coords'][i]
ids = route_output_df['route_ids'][i]
duration = route_output_df['route_duration'][i]
cumulative_route_load = route_output_df['cumulative_route_load'][i]
geometry = route_output_df['route_geometry'][i]
decoded = convert.decode_polyline(geometry)

distance_txt = "<h4> <b>Distance :&nbsp" + "<strong>"+str(cumulative_route_load)+" Km </strong>" +"</h4></b>"
duration_txt = "<h4> <b>Duration :&nbsp" + "<strong>"+str(duration)+" Mins. </strong>" +"</h4></b>"

m = folium.Map(location=start, tiles="Cartodb Positron", zoom_start=12)
folium.GeoJson(decoded).add_child(folium.Popup(distance_txt+duration_txt,max_width=300)).add_to(m)

for each in range(len(coordinates)):
    folium.Marker(
        location=list(coordinates[each][::-1]),
        popup=html_popup_series(order_details[order_details['customerCode']==str(ids[each])]),
        icon=folium.Icon(color="green"),
    ).add_to(m)

m

TypeError: object of type 'NoneType' has no len()

In [ ]:
fig = px.choropleth(df_area_sales_2, 
                    geojson=js, 
                    locations="GID_2", 
                    featureidkey="properties.GID_2", 
                    color='itemTotalWeight',
                    color_continuous_scale="ylgn",  # Set color scale to green
                    range_color=(0, 1540),
                    scope='asia'
                   )

# Add points to the choropleth map
scatter_trace = px.scatter_geo(order_details, 
                               lat='customerLat', 
                               lon='customerLong', 
                               size='totalWeight',
                               color='totalWeight',
                               color_continuous_scale="Reds",  # Set color scale to red
                               opacity=.5,
                              )

scatter_trace.update_traces(marker=dict(color='Orange', size=15, line=dict(width=1, color='White')), selector=dict(mode='markers'))
fig.add_trace(scatter_trace.data[0])

# Update layout for better formatting
fig.update_layout(
    coloraxis_colorbar=dict(title='Weight Scale'),  # Adjust color bar title
    geo=dict(projection_type='mercator'  # Adjust projection type if necessary
            ),
    )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, plot_bgcolor='black')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

NameError: name 'df_area_sales_2' is not defined